In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import csv
from sklearn.linear_model import LinearRegression
#linreg gives straight line
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
#doesnt work idk
from sklearn.ensemble import RandomForestClassifier
#doesnt work
from sklearn.ensemble import RandomForestRegressor
#gives me 1 entire value
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
#gives me 1 value
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import MultiTaskLasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
import time
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from geopy import distance
import preprocess as pre

In [7]:
train = pd.read_csv("ais_train.csv", sep='|')
test = pd.read_csv("ais_test.csv")
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

In [5]:
start = time.time()
#pre.ais_trainCleanup("~/proj2/ML/Henrik/ais_train.csv","ais_clean.csv")
print(time.time()-start)

5.030632019042969e-05


In [4]:
cleandata = pd.read_csv("ais_clean.csv",sep="|")
cleandata['time'] = pd.to_datetime(cleandata['time'])
cleandata.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,dist_s_l,tot_dist,time_s_l,speed,speed_lag,delta_lat,delta_lon,delta_time,last_lon,last_lat
0,2024-01-01 00:17:17,73.9,0.1,12,120,1,2024-12-28 05:00:00,-26.77586,153.23453,61e9f3acb937134a3c4bfe23,...,0.033917,0.033917,361,0.338229,0.0,0.00026,0.00018,361.0,153.23435,-26.77612
1,2024-01-01 00:18:57,150.7,0.1,0,251,0,2024-12-31 04:00:00,38.47387,15.91592,61e9f3cab937134a3c4bff01,...,0.002618,0.002618,159,0.059274,0.0,0.00000,-0.00003,159.0,15.91595,38.47387
2,2024-01-01 00:19:39,56.4,12.9,-16,56,0,2024-12-30 20:00:00,33.90815,130.92404,61e9f3e1b937134a3c4bff59,...,2.085388,2.085388,334,22.477237,0.0,0.00412,0.02200,334.0,130.90204,33.90403
3,2024-01-01 00:20:35,114.5,0.0,0,275,5,2024-01-03 13:30:00,43.44237,-3.82309,clh6aqawa0002gh0zypfa5dut,...,0.005775,0.005775,720,0.028873,0.0,-0.00001,0.00007,720.0,-3.82316,43.44238
4,2024-01-01 00:21:37,210.4,0.0,-5,122,1,2024-12-31 04:00:00,40.68658,29.31613,61e9f3c7b937134a3c4bfedf,...,0.025431,0.025431,360,0.254307,0.0,-0.00021,0.00012,360.0,29.31601,40.68679


In [6]:
#endrer tid og 1hoter vessel
warnings.filterwarnings('ignore')

def preprocess(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    data = pd.get_dummies(data, columns=['vesselId'], drop_first=True)
    return(data)

def timefix(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    return data

def speed(x,time_a,time_b):
    if time_a == time_b:
        return 0
    else:
        return min((x)/(time_b-time_a),50)
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
#Må brukes på enkeltbåt
def deltafier(df):
    data = df.copy()
    data["delta_lat"] = data["latitude"]-data["latitude"].shift(1)
    data["delta_lon"] = (data["longitude"]-data["longitude"].shift(1)+180)%360-180
    data["delta_time"] = timefix(data)["time"]-timefix(data)["time"].shift(1)
    data["last_lon"] = data["longitude"].shift(1)
    data["last_lat"] = data["latitude"].shift(1)
    return data[1:]

In [8]:
start = time.time()
trainingdata5 = cleandata[["delta_time","last_lat","last_lon","speed_lag"]]
goaldata5 = cleandata[["delta_lat","delta_lon"]]
trainingdata5 = trainingdata5.reindex(sorted(trainingdata5.columns), axis=1)
print(time.time()-start)

0.049356937408447266


In [9]:
start = time.time()
model5 = xgb.XGBRegressor()
model5.fit(trainingdata5,goaldata5)
print(time.time()-start)

6.944178581237793


In [10]:
uniquevessel = [i for i in test.copy()['vesselId'].unique()]
bettertraining = cleandata[0:1]
for i in uniquevessel:
    boat = cleandata[cleandata['vesselId'] == i]
    bettertraining.loc[len(bettertraining.index)] = boat.iloc[-1]
bettertraining = bettertraining[1:]
bettertraining.tail()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,dist_s_l,tot_dist,time_s_l,speed,speed_lag,delta_lat,delta_lon,delta_time,last_lon,last_lat
211,2024-04-26 13:50:08,188.2,17.1,2,187,0,2024-04-27 20:00:00,36.66386,125.90680,631e3eb0192150e13fa41628,...,5.644802,42924.871134,630,32.256013,32.862252,-0.05054,-0.00715,630.0,125.91395,36.71440
212,2024-04-30 18:39:21,251.9,14.3,9,253,0,2024-05-03 09:30:00,34.64546,139.47670,61e9f41cb937134a3c4c0091,...,4.161817,55124.359833,558,26.850431,27.964774,-0.02140,-0.03729,558.0,139.51399,34.66686
213,2024-04-30 07:08:07,137.5,4.5,-15,137,0,2024-04-29 14:00:00,32.79268,-79.92353,61e9f39fb937134a3c4bfdcd,...,0.362835,55923.219291,48893,0.026716,0.004151,-0.00214,0.00293,48893.0,-79.92646,32.79482
214,2024-04-21 08:15:03,6.2,16.1,0,5,0,2024-04-19 03:30:00,25.29443,121.26813,6326eed6c46d6a20d22ca319,...,11.355915,20291.132863,1488,27.473988,18.832745,0.09506,-0.04219,1488.0,121.31032,25.19937
215,2024-05-03 22:32:02,76.9,13.8,0,79,0,2024-05-11 22:00:00,1.27494,104.27201,61e9f45ab937134a3c4c021b,...,6.056613,50329.787290,851,25.621396,24.054062,0.01101,0.05331,851.0,104.21870,1.26393


In [12]:
start = time.time()
test5 = test.copy()
uniqueVesselId = test5["vesselId"].unique()
test5_processed_list = []
for id in uniqueVesselId:
    boat = test5[test5["vesselId"] == id].reset_index(drop=True)
    last = bettertraining[bettertraining["vesselId"]==id].reset_index(drop=True)
    boat["delta_time"] = timefix(boat)["time"]-timefix(boat)["time"].shift(1)
    boat.at[0,"delta_time"] = timefix(boat).at[0,"time"]-timefix(last).at[0,"time"]
    boat.at[0,"last_lat"] = last.at[0,"latitude"]
    boat.at[0,"last_lon"] = last.at[0,"longitude"]
    boat.at[0,"speed_lag"] = last.at[0,"speed"]
        #Appending rows to list
    for _, row in boat.iterrows():
        test5_processed_list.append(row.to_dict())

test5_processed = pd.DataFrame(test5_processed_list).sort_values(by="vesselId").reset_index(drop=True)
test5_processed["index"] = test.sort_values(by = ["vesselId","time"]).reset_index()["index"]
test5_processed = test5_processed.sort_values(by = "index").drop(columns="index").reset_index(drop=True)

print(time.time()-start)

5.314585447311401


In [13]:
test5p = test5_processed.copy()
#test5p = pd.merge(test5p,bettertraining.copy(), on = "vesselId",how = "left")
#test5p['time']=test5p['time_x']
#test5p["delta_time"]=test5p["delta_time_x"]
#test5p["last_lat"]=test5p["last_lat_x"]
#test5p["last_lon"]=test5p["last_lon_x"]
test5p = test5p[["last_lat","last_lon","vesselId","time","delta_time","speed_lag"]]
test5p = test5p.sort_values(by=["vesselId","time"])


#test5p = pd.get_dummies(test5p, columns=['navstat'], drop_first=True)

for column in trainingdata5:
    if column not in test5p:
        test5p[column] = False


test5p = test5p.reindex(sorted(test5p.columns), axis=1).reset_index()



test5p.tail()

,index,delta_time,last_lat,last_lon,speed_lag,time,vesselId
51734,5118,1213.0,NaN,NaN,NaN,2024-05-12 22:37:33,clh6aqawa0007gh0z9h6zi9bo
51735,4953,1232.0,NaN,NaN,NaN,2024-05-12 22:58:05,clh6aqawa0007gh0z9h6zi9bo
51736,4845,1215.0,NaN,NaN,NaN,2024-05-12 23:18:20,clh6aqawa0007gh0z9h6zi9bo
51737,4728,1261.0,NaN,NaN,NaN,2024-05-12 23:39:21,clh6aqawa0007gh0z9h6zi9bo
51738,4599,1182.0,NaN,NaN,NaN,2024-05-12 23:59:03,clh6aqawa0007gh0z9h6zi9bo


In [14]:
#lat,lon
start = time.time()
worker = test5p.copy().drop(columns="index")

def predi(data,vid,tim):
    work = data.copy().reset_index(drop=True)
    for i in range(len(work)-1):
        pred = model5.predict(work[i:i+1])[0]
        latlon = [work.at[i,"last_lat"]+pred[0],(work.at[i,"last_lon"]+pred[1]+180)%360-180]
        dist = distance.geodesic((work.at[i,"last_lat"],work.at[i,"last_lon"]),(latlon[0],latlon[1])).km
        work.at[i+1,"last_lat"]=latlon[0]
        work.at[i+1,"last_lon"]=latlon[1]
        work.at[i+1,"speed_lag"] = dist/work.at[i+1,"delta_time"]*3600
    last = model5.predict(work[-1:])[0]
    work["latitude"]=work["last_lat"].shift(-1)
    work["longitude"]=work["last_lon"].shift(-1)
    work.at[len(work)-1,"latitude"]=work.at[len(work)-1,"last_lat"]+last[0]
    work.at[len(work)-1,"longitude"]=(work.at[len(work)-1,"last_lon"]+last[1]+180)%360-180
    work["vesselId"]=id
    work["time"]=tim
    return work

uniqueVessel = worker["vesselId"].unique()
dog_list = []
for id in uniqueVesselId:
    #Isolate a single boat and clean
    boat = worker[worker["vesselId"] == id].drop(columns=["vesselId"])
    tim = boat["time"]
    boat = predi(boat.drop(columns="time"),id,tim)
    for _, row in boat.iterrows():
        dog_list.append(row.to_dict())

worker = pd.DataFrame(dog_list).sort_values(by=["vesselId","time"]).reset_index(drop=True)

worker["index"]=test5p["index"]
worker = worker.sort_values(by=["index"]).reset_index(drop=True)

print(time.time()-start)
worker.tail()

167.49267554283142


,delta_time,last_lat,last_lon,speed_lag,latitude,longitude,vesselId,time,index
51734,1220.0,30.809282,-62.052694,0.392437,30.809494,-62.054062,61e9f3a8b937134a3c4bfdf3,NaT,51734
51735,1219.0,29.937311,-131.366754,3.132236,29.930857,-131.358645,61e9f3b4b937134a3c4bfe77,NaT,51735
51736,1311.0,24.754933,-142.799315,28.797136,24.771326,-142.802556,61e9f46cb937134a3c4c02b7,NaT,51736
51737,1217.0,57.820518,18.625953,1.017500,57.817467,18.625052,61e9f465b937134a3c4c0269,NaT,51737
51738,1258.0,56.150003,10.236224,0.358639,56.149286,10.237779,61e9f3adb937134a3c4bfe39,NaT,51738


In [15]:
ramg = [i for i in range(51739)]
#res2 = [ramg,[(i[1]+180)%360-180 for i in res],[(i[0]+90)%180-90 for i in res]]
#res3 = [ramg,[i[0] for i in res2],[i[0] for i in res1]]
res3 = [ramg,worker["longitude"],worker["latitude"]]
with open('res.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(["ID","longitude_predicted","latitude_predicted"])
    write.writerows(zip(*res3))
df = pd.read_csv('res.csv')
df.to_csv('res.csv', index=False)